In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("tree").getOrCreate()
data = spark.read.format("csv").load("dbfs:/FileStore/shared_uploads/necmettinceylan@hotmail.com/College.csv",header=True,
                                    inferSchema=True)

data.printSchema()

root
-- School: string (nullable = true)
-- Private: string (nullable = true)
-- Apps: integer (nullable = true)
-- Accept: integer (nullable = true)
-- Enroll: integer (nullable = true)
-- Top10perc: integer (nullable = true)
-- Top25perc: integer (nullable = true)
-- F_Undergrad: integer (nullable = true)
-- P_Undergrad: integer (nullable = true)
-- Outstate: integer (nullable = true)
-- Room_Board: integer (nullable = true)
-- Books: integer (nullable = true)
-- Personal: integer (nullable = true)
-- PhD: integer (nullable = true)
-- Terminal: integer (nullable = true)
-- S_F_Ratio: double (nullable = true)
-- perc_alumni: integer (nullable = true)
-- Expend: integer (nullable = true)
-- Grad_Rate: integer (nullable = true)

In [0]:
data.head(1)

Out[2]: [Row(School='Abilene Christian University', Private='Yes', Apps=1660, Accept=1232, Enroll=721, Top10perc=23, Top25perc=52, F_Undergrad=2885, P_Undergrad=537, Outstate=7440, Room_Board=3300, Books=450, Personal=2200, PhD=70, Terminal=78, S_F_Ratio=18.1, perc_alumni=12, Expend=7041, Grad_Rate=60)]

In [0]:
from pyspark.ml.feature import VectorAssembler
data.columns

Out[3]: ['School',
 'Private',
 'Apps',
 'Accept',
 'Enroll',
 'Top10perc',
 'Top25perc',
 'F_Undergrad',
 'P_Undergrad',
 'Outstate',
 'Room_Board',
 'Books',
 'Personal',
 'PhD',
 'Terminal',
 'S_F_Ratio',
 'perc_alumni',
 'Expend',
 'Grad_Rate']

In [0]:
assembler = VectorAssembler( inputCols=[ 'Apps',
 'Accept',
 'Enroll',
 'Top10perc',
 'Top25perc',
 'F_Undergrad',
 'P_Undergrad',
 'Outstate',
 'Room_Board',
 'Books',
 'Personal',
 'PhD',
 'Terminal',
 'S_F_Ratio',
 'perc_alumni',
 'Expend',
 'Grad_Rate' ],outputCol="features")

In [0]:
output = assembler.transform(data)


In [0]:
from pyspark.ml.feature import StringIndexer
indexer =StringIndexer( inputCol="Private", outputCol="PrivateIndex" )

In [0]:
output_fixed = indexer.fit(output).transform(output)

In [0]:
output_fixed.printSchema()

root
-- School: string (nullable = true)
-- Private: string (nullable = true)
-- Apps: integer (nullable = true)
-- Accept: integer (nullable = true)
-- Enroll: integer (nullable = true)
-- Top10perc: integer (nullable = true)
-- Top25perc: integer (nullable = true)
-- F_Undergrad: integer (nullable = true)
-- P_Undergrad: integer (nullable = true)
-- Outstate: integer (nullable = true)
-- Room_Board: integer (nullable = true)
-- Books: integer (nullable = true)
-- Personal: integer (nullable = true)
-- PhD: integer (nullable = true)
-- Terminal: integer (nullable = true)
-- S_F_Ratio: double (nullable = true)
-- perc_alumni: integer (nullable = true)
-- Expend: integer (nullable = true)
-- Grad_Rate: integer (nullable = true)
-- features: vector (nullable = true)
-- PrivateIndex: double (nullable = false)

In [0]:
my_final_data = output_fixed.select("features","PrivateIndex")

In [0]:
train_data,test_data =  my_final_data.randomSplit([0.7,0.2])


In [0]:
from pyspark.ml.classification import DecisionTreeClassifier,GBTClassifier,RandomForestClassifier

from pyspark.ml import Pipeline
dtc = DecisionTreeClassifier(labelCol="PrivateIndex", featuresCol="features")
rfc = RandomForestClassifier(labelCol="PrivateIndex", featuresCol="features")
gbt = GBTClassifier(labelCol="PrivateIndex", featuresCol="features")

dtc_model = dtc.fit(train_data)
rfc_model = rfc.fit(train_data)
gbt_model = gbt.fit(train_data)

dtc_preds = dtc_model.transform(test_data)
rfc_preds = rfc_model.transform(test_data)
gbt_preds = gbt_model.transform(test_data)


In [0]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

my_binary_eval = BinaryClassificationEvaluator(labelCol="PrivateIndex")



In [0]:
print( my_binary_eval.evaluate(dtc_preds) )
print( my_binary_eval.evaluate(rfc_preds) )
print( my_binary_eval.evaluate(gbt_preds) )

0.828526645768025
0.9749216300940439
0.9665360501567402

In [0]:
gbt_preds.printSchema()

root
-- features: vector (nullable = true)
-- PrivateIndex: double (nullable = false)
-- rawPrediction: vector (nullable = true)
-- probability: vector (nullable = true)
-- prediction: double (nullable = false)

In [0]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

acc_eval = MulticlassClassificationEvaluator(labelCol="PrivateIndex",
                                            metricName="accuracy")



In [0]:
rfc_acc = acc_eval.evaluate(rfc_preds)

rfc_acc

Out[22]: 0.9298245614035088